In [1]:
import openai
import httpx
import os

deployment_name = "rl_agent"
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = "https://smerrill-openai-test.openai.azure.com/"
openai.api_version = "2023-09-15-preview"

### General Helper Functions

In [2]:
# Defining a function to send the prompt to the model
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()

### Define Tools LLM can interact with

In [3]:
def calculate(what):
    return eval(what)

In [4]:
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


In [5]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

### System Message to provide

In [6]:
base_system_message = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.
You can try refining a wikipedia search to something more specific if the original query doesn't answer your question

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
"""

system_message = f"{base_system_message.strip()}"
print(system_message)

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.
You can try refining a wikipedia search to something more specific if the original query doesn't answer your question

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then 

#### Question Query

In [7]:
# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "Who is the main character in Quentin tarantino's second film?"

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

In [8]:
max_response_tokens = 500

response = send_message(messages, deployment_name, max_response_tokens)
messages.append({"role": "assistant", "content": response})

#### Action Selection

In [9]:
print(response)

Thought: I know that Quentin Tarantino's second film is Pulp Fiction, but I'm not sure who the main character is.
Action: wikipedia: Pulp Fiction
PAUSE


#### Observation from selected action

In [10]:
obs = 'Observation: ' + wikipedia("Pulp Fiction")
messages.append({"role": "user", "name":"example_user", "content": obs})
print(obs)

Observation: <span class="searchmatch">Pulp</span> <span class="searchmatch">Fiction</span> is a 1994 American independent crime film written and directed by Quentin Tarantino from a story he conceived with Roger Avary. It tells four


#### New action from response

In [11]:
response = send_message(messages, deployment_name, max_response_tokens)

In [12]:
print(response)

Answer: The main character in Quentin Tarantino's second film, Pulp Fiction, is debatable as there are multiple main characters. However, the most prominent characters are Vincent Vega and Jules Winnfield, played by John Travolta and Samuel L. Jackson, respectively.
